# Set configuration

You need to login to a server and fill the following variables:

In [ ]:
from IPython.core.display import display
import ipywidgets as widgets

session_widget = widgets.Text(placeholder='PHPSESSID', description='Session ID')
server_widget = widgets.Text(value='s1.railnation.de', description='Server URL')

display(session_widget, server_widget)

In [ ]:
session_id = session_widget.value
server_url = server_widget.value

%load_ext autoreload
%autoreload 2

In [ ]:
# get user ID
from server import ServerCaller

api = ServerCaller(server_url, session_id)
server = api.call('ServerInfoInterface', 'getInfo', None)
user_id = server['Infos']['activeUser']
print(f'Logged in to {server["Body"]["worldName"]}')

In [ ]:
# association analysis
assoc_info = api.call('CorporationInterface', 'getOtherMemberBuildings',
                      ["00000000-0000-0000-0000-000000000000"])
# 7: Hotel, 8: Restaurant, 9: Mall
print(f"Info for {len(assoc_info['Body'])} members")

In [ ]:
from time import sleep
prestiges = {}
for user_id, member in assoc_info['Body'].items():
    prestiges[user_id] = {}
    prestiges[user_id]['overview'] = api.call('BudgetInterface', 'getPrestigeHistoryDetails',
                      [user_id, 9])['Body']['balance']
    prestiges[user_id]['cities'] = api.call('BudgetInterface', 'getPrestigeHistoryDetails',
                      [user_id, 9, '0', '1'])['Body']['balance']
    prestiges[user_id]['landmarks'] = api.call('BudgetInterface', 'getPrestigeHistoryDetails',
                      [user_id, 9, '0', '2'])['Body']['balance']
    sleep(1)
    print('.')
len(prestiges)

In [ ]:
user_ids = list(prestiges.keys())
profiles = api.call('ProfileInterface', 'getVCard', [user_ids], short_call=1236)['Body']
len(profiles)

In [ ]:
import csv

cities = set(city['type'] for user_id, prestige in prestiges.items() for city in prestige['cities'] )
landmarks = set(landmark['type'] for user_id, prestige in prestiges.items() for landmark in prestige['landmarks'] )

with open('prestige.csv', 'w', newline='') as csvfile:
    fieldnames = ['member', 'transports', 'invests', 'trainstation', 'other', 'competition', 'medal', '-cities-'] + list(cities) + ['-landmarks-'] + list(landmarks)
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for user_id, prestige in prestiges.items():
        overview_dict = {
            'transports': [row for row in prestige['overview'] if row['type'] == '0'],
             'invests': [row for row in prestige['overview'] if row['type'] == '3'],
             'trainstation': [row for row in prestige['overview'] if row['type'] == '4'],
             'other': [row for row in prestige['overview'] if row['type'] == '6'],
             'competition': [row for row in prestige['overview'] if row['type'] == '7'],
             'medal': [row for row in prestige['overview'] if row['type'] == '8'],
        }
        write_dict = {'member': profiles[user_id]['userName']}
        for key, value in overview_dict.items():
            if len(value) > 0:
                write_dict[key] = value[0]['prestige']

        for row in prestige['cities']:
            write_dict[row['type']] = row['prestige']
        for row in prestige['landmarks']:
            write_dict[row['type']] = row['prestige']

        writer.writerow(write_dict)
len(prestiges)